In [2]:
!pip install twilio

   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.9 MB ? eta -:--:--
   ---------------- ----------------------- 0.8/1.9 MB 1.5 MB/s eta 0:00:01
   ---------------------- ----------------- 1.0/1.9 MB 1.7 MB/s eta 0:00:01
   --------------------------------- ------ 1.6/1.9 MB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 2.0 MB/s eta 0:00:00


In [5]:
import streamlit as st
# ONLY KEEP THIS ONE ▼
st.set_page_config(page_title="Rakshini Navigator", layout="centered")

# Then all other imports
import pandas as pd
import networkx as nx
from datetime import datetime
from twilio.rest import Client

# ---------------- Twilio Config ----------------
TWILIO_SID = "your_account_sid"
TWILIO_AUTH_TOKEN = "your_auth_token"
TWILIO_PHONE = "your_twilio_number"  # e.g., +1234567890
emergency_phone_numbers = ["+919999999999", "+911234567890"]

# ---------------- Load Graph from CSV ----------------
@st.cache_data
def load_graph_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    graph = {}
    for _, row in df.iterrows():
        src, dst, risk = row["Source"], row["Destination"], row["DangerScore"]
        graph.setdefault(src, {})[dst] = risk
        graph.setdefault(dst, {})[src] = risk
    return graph, list(set(df["Source"]).union(set(df["Destination"])))

graph, all_locations = load_graph_from_csv("graph_data.csv")

# ---------------- Dijkstra Algorithm ----------------
def dijkstra(graph, start, end):
    G = nx.Graph()
    for u in graph:
        for v, w in graph[u].items():
            G.add_edge(u, v, weight=w)
    try:
        path = nx.dijkstra_path(G, start, end, weight='weight')
        score = sum(graph[path[i]][path[i+1]] for i in range(len(path)-1))
        return path, score
    except:
        return None, None

# ---------------- Twilio Alert Sender ----------------
def send_emergency_alert(user_name, location):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    message_body = f"""
🚨 EMERGENCY ALERT 🚨
Name: {user_name}
Location: {location}
Time: {timestamp}
Please respond immediately!
"""

    try:
        client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)
        for number in emergency_phone_numbers:
            message = client.messages.create(
                body=message_body,
                from_=TWILIO_PHONE,
                to=number
            )
            print(f"Alert sent to {number}: SID {message.sid}")
        return True
    except Exception as e:
        print("Error sending SMS:", e)
        return False

# ---------------- UI ----------------
st.title("🛡 Rakshini - Safe Route Navigator for Women")

st.markdown("Enter your *source* and *destination* to find the safest path:")

col1, col2 = st.columns(2)
with col1:
    source = st.selectbox("Source Location", all_locations)
with col2:
    destination = st.selectbox("Destination Location", all_locations)

if st.button("🔍 Find Safest Route"):
    if source == destination:
        st.warning("Source and destination must be different.")
    else:
        path, risk = dijkstra(graph, source, destination)
        if path:
            st.success("Safest Path: " + " → ".join(path))
            st.info(f"Total Risk Score: {risk}")
        else:
            st.error("No valid path found between selected locations.")

# ---------------- Emergency Section ----------------
st.markdown("---")
st.header("🚨 Emergency Alert")

# Simulated User Details
user_name = "Ananya Sharma"
current_location = source  # Using selected source as current location

if st.button("🚨 Send Emergency Alert via SMS"):
    success = send_emergency_alert(user_name, current_location)
    if success:
        st.warning("🚨 SMS alert successfully sent to emergency contacts.")
    else:
        st.error("❌ Failed to send SMS. Please check your Twilio setup.")

# ---------------- Contacts Display ----------------
st.markdown("---")
st.header("👥 Your Trusted Contacts")
for number in emergency_phone_numbers:
    st.write(f"📞 {number}")

2025-05-22 21:02:00.235 No runtime found, using MemoryCacheStorageManager
